<a href="https://colab.research.google.com/github/aarohigorle2005/Data_Science_Lab/blob/main/Experiment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implement the basic Minimax algorithm for two-player deterministic games.

## Minimax Algorithm Implementation

Minimax is a decision rule used in game theory to minimize the possible loss for a worst-case (maximum loss) scenario. When used in AI, it helps an agent make optimal moves in a two-player, zero-sum game, assuming the opponent also plays optimally.

### Key Concepts:

*   **Players:** Typically a 'Maximizing Player' (trying to get the highest score) and a 'Minimizing Player' (trying to get the lowest score).
*   **Game Tree:** Minimax explores a tree of possible game states, where each node represents a board configuration and edges represent moves.
*   **Evaluation Function:** Assigns a numerical value to a terminal game state (win, lose, draw).
*   **Recursion:** The algorithm works by recursively looking ahead to possible future moves.

We'll implement Minimax for **Tic-Tac-Toe**.

### Tic-Tac-Toe Game Setup

First, let's set up the basic functions to play Tic-Tac-Toe: representing the board, printing it, checking for wins/draws, and handling moves.

In [19]:
# Game board representation (a 3x3 grid, flattened into a list of 9 elements)
# Each element can be ' ' (empty), 'X' (human player), or 'O' (AI player)
board = [' ' for _ in range(9)] # Initialize all 9 positions as empty

def print_board(board):
    """
    Prints the Tic-Tac-Toe board to the console in a readable 3x3 format.
    """
    print(f" {board[0]} | {board[1]} | {board[2]} ")
    print("-----------")
    print(f" {board[3]} | {board[4]} | {board[5]} ")
    print("-----------")
    print(f" {board[6]} | {board[7]} | {board[8]} ")

def check_win(board, player):
    """
    Checks if the given 'player' ('X' or 'O') has won the game.
    A win occurs if a player has three of their marks in a row, column, or diagonal.
    """
    win_conditions = [
        [0, 1, 2], [3, 4, 5], [6, 7, 8], # Rows: Top, Middle, Bottom
        [0, 3, 6], [1, 4, 7], [2, 5, 8], # Columns: Left, Middle, Right
        [0, 4, 8], [2, 4, 6]            # Diagonals
    ]
    for condition in win_conditions:
        # Check if all three positions in the current winning condition match the player's mark
        if all(board[i] == player for i in condition):
            return True # Player has won
    return False # No win found for this player

def check_draw(board):
    """
    Checks if the game is a draw. A draw occurs if there are no empty spaces left
    and neither player has won.
    """
    # ' ' not in board checks if there are any empty spots left
    # The other two conditions ensure no one has won yet
    return ' ' not in board and not check_win(board, 'X') and not check_win(board, 'O')

def get_empty_cells(board):
    """
    Returns a list of indices (0-8) of all empty cells on the board.
    These are the available moves.
    """
    return [i for i, spot in enumerate(board) if spot == ' ']

### Minimax Algorithm Core Logic

Now for the Minimax algorithm itself. This function will recursively simulate moves and choose the best one. For Tic-Tac-Toe:

*   **Maximizing Player (AI - 'O'):** Tries to maximize its score (10 for a win, 0 for a draw, -10 for a loss).
*   **Minimizing Player (Human - 'X'):** Tries to minimize the AI's score (i.e., maximize its own chances of winning or drawing).

In [20]:
def minimax(board, depth, is_maximizing_player):
    """
    Implements the Minimax algorithm to find the best possible score from a given board state.
    It recursively explores the game tree.

    Args:
        board (list): The current state of the Tic-Tac-Toe board.
        depth (int): The current depth in the game tree (number of moves simulated).
                     Used to prefer quicker wins/delay losses.
        is_maximizing_player (bool): True if the current turn is for the maximizing player (AI 'O'),
                                     False if it's for the minimizing player (Human 'X').

    Returns:
        int: The best score that the current player can achieve from this state.
    """

    # --- Base Cases: When the game ends, we return a score ---
    if check_win(board, 'O'): # If AI 'O' wins
        return 10 - depth # Award +10, but subtract depth (to prefer winning faster)
    if check_win(board, 'X'): # If Human 'X' wins
        return -10 + depth # Award -10, but add depth (to prefer delaying loss)
    if check_draw(board): # If it's a draw
        return 0 # Neutral score

    # Safety check: If there are no more moves and no win/draw detected, it's a draw
    if not get_empty_cells(board):
        return 0

    # --- Recursive Cases: Game is not over, so simulate moves ---
    if is_maximizing_player: # AI's turn (Maximizing player)
        best_score = -float('inf') # Start with a very low score, AI wants to find a higher one
        for move in get_empty_cells(board): # Consider all possible empty cells
            board[move] = 'O' # Make the AI's move (simulate)
            # Recursively call minimax for the opponent (minimizing player) with increased depth
            score = minimax(board, depth + 1, False)
            board[move] = ' ' # Undo the move (important for backtracking and trying other paths)
            best_score = max(best_score, score) # AI chooses the move that leads to the maximum score
        return best_score
    else: # Human's turn (Minimizing player)
        best_score = float('inf') # Start with a very high score, Human wants to find a lower one for AI
        for move in get_empty_cells(board): # Consider all possible empty cells
            board[move] = 'X' # Make the Human's move (simulate)
            # Recursively call minimax for the AI (maximizing player) with increased depth
            score = minimax(board, depth + 1, True)
            board[move] = ' ' # Undo the move (backtracking)
            best_score = min(best_score, score) # Human chooses the move that leads to the minimum score for AI
        return best_score

def find_best_move(board):
    """
    This function is called by the AI to determine its next optimal move.
    It iterates through all possible moves, uses minimax to evaluate each, and picks the best one.
    """
    best_score = -float('inf') # Initialize with a very low score
    best_move = -1 # Initialize with no move selected

    # Iterate through all available empty cells to consider as potential moves
    for move in get_empty_cells(board):
        board[move] = 'O' # Make a temporary move for the AI
        # Call minimax to evaluate this move. The next turn will be for the human (minimizing player).
        # Start with depth 0 for the initial call.
        score = minimax(board, 0, False)
        board[move] = ' ' # Undo the temporary move (crucial for trying other moves)

        # If this move leads to a better score than previously found, update best_score and best_move
        if score > best_score:
            best_score = score
            best_move = move

    return best_move # Return the index of the best move for the AI

### Play Tic-Tac-Toe with Minimax AI!

Now, let's put it all together. You'll play as 'X' and the AI will play as 'O'. The AI will use the Minimax algorithm to make its moves.

In [21]:
# Reset the board for a new game
current_board = [' ' for _ in range(9)]

print("Welcome to Tic-Tac-Toe with Minimax AI!")
print("You are 'X', the AI is 'O'.")
print("Enter your move by typing a number 0-8 for the cell you want to play.")

# Display the initial empty board
print_board(current_board)

# Main game loop
while True:
    # --- Human Player's turn (X) ---
    human_move = None
    while human_move is None: # Loop until a valid move is entered
        try:
            move_input = input("Enter your move (0-8): ")
            move = int(move_input)
            # Check if the move is within bounds (0-8) and the cell is empty
            if 0 <= move <= 8 and current_board[move] == ' ':
                human_move = move # Valid move found
            else:
                print("Invalid move. Please choose an empty cell between 0 and 8.")
        except ValueError:
            print("Invalid input. Please enter a number.")

    # Place human's mark on the board
    current_board[human_move] = 'X'
    print_board(current_board)

    # Check for game end conditions after human's move
    if check_win(current_board, 'X'):
        print("Congratulations! You win!")
        break # End game loop
    if check_draw(current_board):
        print("It's a draw!")
        break # End game loop

    # --- AI Player's turn (O) ---
    print("AI is making a move...")
    # AI uses the find_best_move function (powered by Minimax) to get its optimal move
    ai_move = find_best_move(current_board)
    current_board[ai_move] = 'O' # Place AI's mark on the board
    print_board(current_board)

    # Check for game end conditions after AI's move
    if check_win(current_board, 'O'):
        print("AI wins! Better luck next time.")
        break # End game loop
    if check_draw(current_board):
        print("It's a draw!")
        break # End game loop


Welcome to Tic-Tac-Toe with Minimax AI!
You are 'X', the AI is 'O'.
Enter your move by typing a number 0-8 for the cell you want to play.
   |   |   
-----------
   |   |   
-----------
   |   |   
Enter your move (0-8): 6
   |   |   
-----------
   |   |   
-----------
 X |   |   
AI is making a move...
   |   |   
-----------
   | O |   
-----------
 X |   |   
Enter your move (0-8): 4
Invalid move. Please choose an empty cell between 0 and 8.
Enter your move (0-8): 7
   |   |   
-----------
   | O |   
-----------
 X | X |   
AI is making a move...
   |   |   
-----------
   | O |   
-----------
 X | X | O 
Enter your move (0-8): 3
   |   |   
-----------
 X | O |   
-----------
 X | X | O 
AI is making a move...
 O |   |   
-----------
 X | O |   
-----------
 X | X | O 
AI wins! Better luck next time.


### How to play:

1.  **Run all the code cells** sequentially.
2.  The game board will be displayed.
3.  When prompted, **enter a number between 0 and 8** corresponding to the cell where you want to place your 'X'.
4.  The AI ('O') will then make its optimal move.
5.  Continue playing until there's a winner or a draw!

Implementation of A* algorithm.

## A* Search Algorithm

A* (pronounced "A star") is a graph traversal and path search algorithm, which is often used in many fields of computer science due to its completeness, optimality, and optimal efficiency. It is an informed search algorithm, meaning it uses a heuristic function to guide its search.

We will demonstrate A* on a simple grid-based maze, where:
*   `S` represents the Start point.
*   `E` represents the End (Goal) point.
*   `#` represents an obstacle (impassable).
*   `.` represents an open path.

In [9]:
import heapq # This module implements the heap queue algorithm, useful for priority queues.

# Define our maze as a list of lists (a 2D grid)
# 'S' is Start, 'E' is End, '#' is obstacle, '.' is open path
maze = [
    ['S', '.', '.', '#', '.', '.', '.'],
    ['.', '#', '.', '.', '.', '#', '.'],
    ['.', '#', '.', '#', '.', '.', '.'],
    ['.', '.', '.', '#', '#', '.', '.'],
    ['#', '.', '.', '.', '.', '.', 'E']
]

def print_maze(maze, path=[]):
    """
    Prints the maze, highlighting the path taken (if any).
    Args:
        maze (list of lists): The 2D grid representing the maze.
        path (list of tuples): A list of (row, column) coordinates forming the path.
    """
    for r_idx, row in enumerate(maze):
        for c_idx, char in enumerate(row):
            # If the current cell is part of the path (excluding Start/End for 'X' marking)
            if (r_idx, c_idx) in path:
                print('X', end=' ') # Mark path cells with 'X'
            else:
                print(char, end=' ') # Print the original maze character
        print() # Move to the next line after each row

print("Initial Maze:")
print_maze(maze) # Display the initial state of our maze

Initial Maze:
S . . # . . . 
. # . . . # . 
. # . # . . . 
. . . # # . . 
# . . . . . E 


### Heuristic Function (Manhattan Distance)

For a grid, a common heuristic is the Manhattan distance (L1 norm), which calculates the sum of the absolute differences of their coordinates. This estimates the number of moves needed to reach the goal, ignoring obstacles.

In [14]:
def heuristic(position_a, position_b):
    """
    Calculates the Manhattan distance between two points on the grid.
    This serves as our heuristic (h-score) for the A* algorithm.

    Args:
        position_a (tuple): (row, column) coordinates of the first point.
        position_b (tuple): (row, column) coordinates of the second point (typically the goal).

    Returns:
        int: The Manhattan distance.
    """
    return abs(position_a[0] - position_b[0]) + abs(position_a[1] - position_b[1])

### A* Algorithm Implementation

This implementation finds the shortest path from the start node 'S' to the end node 'E' in the maze, avoiding obstacles '#'.

In [15]:
def a_star(maze):
    """
    Implements the A* pathfinding algorithm to find the shortest path
    from 'S' to 'E' in a given maze.

    Args:
        maze (list of lists): The 2D grid representing the maze.

    Returns:
        list: A list of (row, column) coordinates representing the shortest path,
              or a string message if no path is found or S/E are missing.
    """
    rows, cols = len(maze), len(maze[0])
    start_node, end_node = None, None

    # First, find the start 'S' and end 'E' nodes in the maze
    for r in range(rows):
        for c in range(cols):
            if maze[r][c] == 'S':
                start_node = (r, c)
            elif maze[r][c] == 'E':
                end_node = (r, c)

    # Check if both start and end nodes were found
    if not start_node or not end_node:
        return "Error: Start or End node not found in maze."

    # The 'open_set' is a priority queue that stores nodes to be evaluated.
    # It stores tuples: (f_score, node_coordinates)
    # We use heapq to efficiently get the node with the lowest f_score.
    open_set = []
    # Add the start node to the open set with an initial f_score (g=0 + h(start, end))
    heapq.heappush(open_set, (0, start_node))

    # 'came_from' dictionary helps reconstruct the path later.
    # It stores: `child_node: parent_node`
    came_from = {}

    # 'g_score' is the cost from the start node to the current node.
    # Initialize all g_scores to infinity, except for the start node which is 0.
    g_score = { (r, c): float('inf') for r in range(rows) for c in range(cols) }
    g_score[start_node] = 0

    # 'f_score' is the total estimated cost from start to end through this node (g + h).
    # Initialize all f_scores to infinity, except for the start node.
    f_score = { (r, c): float('inf') for r in range(rows) for c in range(cols) }
    f_score[start_node] = heuristic(start_node, end_node)

    # Main A* algorithm loop
    while open_set:
        # Get the node with the lowest f_score from the open set
        # heapq.heappop removes and returns the smallest item from the heap.
        current_f_score, current_coordinates = heapq.heappop(open_set)

        # If we reached the end node, we've found the shortest path!
        if current_coordinates == end_node:
            path = []
            # Reconstruct the path by backtracking from the end node using 'came_from'
            while current_coordinates in came_from:
                path.append(current_coordinates)
                current_coordinates = came_from[current_coordinates]
            path.append(start_node) # Add the start node to the beginning of the path
            return path[::-1] # Reverse the path to get it from start to end

        # Explore neighbors (up, down, left, right) of the current node
        # dr, dc represent changes in row and column indices
        for dr, dc in [(0, 1), (0, -1), (1, 0), (-1, 0)]: # Right, Left, Down, Up moves
            neighbor_coordinates = (current_coordinates[0] + dr, current_coordinates[1] + dc)

            # Check if the neighbor is within maze bounds and not an obstacle
            if (
                0 <= neighbor_coordinates[0] < rows and
                0 <= neighbor_coordinates[1] < cols and
                maze[neighbor_coordinates[0]][neighbor_coordinates[1]] != '#' # Cannot move through obstacles
            ):
                # Calculate the tentative g-score for this neighbor.
                # Assuming each step has a cost of 1 (uniform cost).
                tentative_g_score = g_score[current_coordinates] + 1

                # If this new path to the neighbor is shorter than any previously found path
                if tentative_g_score < g_score[neighbor_coordinates]:
                    # Update path information
                    came_from[neighbor_coordinates] = current_coordinates
                    g_score[neighbor_coordinates] = tentative_g_score

                    # Update f-score for the neighbor (g-score + heuristic)
                    f_score[neighbor_coordinates] = tentative_g_score + heuristic(neighbor_coordinates, end_node)

                    # Add the neighbor to the open set for future evaluation
                    # (Only if it's not already in the open set with a better path, which heapq handles implicitly)
                    heapq.heappush(open_set, (f_score[neighbor_coordinates], neighbor_coordinates))

    # If the loop finishes and we haven't reached the end node, it means no path was found.
    return "No path found from Start to End."

### Demonstrate A* Pathfinding

Let's find the shortest path from 'S' to 'E' in our defined maze and visualize it.

In [12]:
# Call the a_star function with our defined maze to find the shortest path
shortest_path = a_star(maze)

# Check the result of the A* algorithm
if isinstance(shortest_path, list):
    print("Shortest path found! Here's the maze with the path marked:")
    # Print the maze, marking the path (excluding start and end for 'X' for clarity)
    # We print the maze with 'X's for intermediate path steps.
    print_maze(maze, path=shortest_path[1:-1])
    print(f"\nPath coordinates (row, column) from S to E: {shortest_path}")
else:
    # If A* returned a string, it means an error occurred or no path was found
    print(shortest_path)

Shortest path found! Here's the maze with the path marked:
S X X # . . . 
. # X X X # . 
. # . # X X X 
. . . # # . X 
# . . . . . E 

Path coordinates (row, column) from S to E: [(0, 0), (0, 1), (0, 2), (1, 2), (1, 3), (1, 4), (2, 4), (2, 5), (2, 6), (3, 6), (4, 6)]


### How to use:

1.  **Modify the `maze` list** in the first code cell to define your own grid structure. Use 'S' for start, 'E' for end, '#' for obstacles, and '.' for clear paths.
2.  **Run the cells** sequentially.
3.  The output will display the initial maze and then the maze with the shortest path marked by 'X's, along with the list of coordinates forming the path.

 Demonstrate Breadth First Search and Depth First Search

## Graph Representation

First, let's define a simple graph using an adjacency list. You can modify this dictionary to represent your desired graph.

In [2]:
# Define the graph using an adjacency list
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

print("Graph representation:", graph)

Graph representation: {'A': ['B', 'C'], 'B': ['A', 'D', 'E'], 'C': ['A', 'F'], 'D': ['B'], 'E': ['B', 'F'], 'F': ['C', 'E']}


## Breadth-First Search (BFS)

BFS explores the graph layer by layer, visiting all neighbors at the current depth before moving to the next depth level. It typically uses a queue data structure.

In [3]:
from collections import deque

def bfs(graph, start_node):
    visited = set()  # Set to keep track of visited nodes
    queue = deque([start_node])  # Initialize a queue with the start node
    bfs_traversal = []  # List to store the BFS traversal order

    visited.add(start_node)

    while queue:
        current_node = queue.popleft()  # Dequeue a node
        bfs_traversal.append(current_node)

        for neighbor in graph[current_node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)  # Enqueue unvisited neighbors
    return bfs_traversal

# Demonstrate BFS
start_node_bfs = 'A'
print(f"BFS Traversal starting from {start_node_bfs}: {bfs(graph, start_node_bfs)}")

start_node_bfs_input = input("Enter a starting node for BFS (e.g., A, B, C): ").upper()
if start_node_bfs_input in graph:
    print(f"BFS Traversal starting from {start_node_bfs_input}: {bfs(graph, start_node_bfs_input)}")
else:
    print(f"Node '{start_node_bfs_input}' not found in the graph.")

BFS Traversal starting from A: ['A', 'B', 'C', 'D', 'E', 'F']
Enter a starting node for BFS (e.g., A, B, C): A
BFS Traversal starting from A: ['A', 'B', 'C', 'D', 'E', 'F']


## Depth-First Search (DFS)

DFS explores as far as possible along each branch before backtracking. It typically uses a stack data structure (or recursion).

In [4]:
def dfs(graph, start_node):
    visited = set()  # Set to keep track of visited nodes
    stack = [start_node]  # Initialize a stack with the start node
    dfs_traversal = []  # List to store the DFS traversal order

    while stack:
        current_node = stack.pop()  # Pop a node from the stack

        if current_node not in visited:
            visited.add(current_node)
            dfs_traversal.append(current_node)

            # Push unvisited neighbors onto the stack (in reverse order for consistent output)
            # This ensures that neighbors are visited in a consistent order (e.g., alphabetical if sorted)
            for neighbor in sorted(graph[current_node], reverse=True):
                if neighbor not in visited:
                    stack.append(neighbor)
    return dfs_traversal

# Demonstrate DFS
start_node_dfs = 'A'
print(f"DFS Traversal starting from {start_node_dfs}: {dfs(graph, start_node_dfs)}")

start_node_dfs_input = input("Enter a starting node for DFS (e.g., A, B, C): ").upper()
if start_node_dfs_input in graph:
    print(f"DFS Traversal starting from {start_node_dfs_input}: {dfs(graph, start_node_dfs_input)}")
else:
    print(f"Node '{start_node_dfs_input}' not found in the graph.")

DFS Traversal starting from A: ['A', 'B', 'D', 'E', 'F', 'C']
Enter a starting node for DFS (e.g., A, B, C): A
DFS Traversal starting from A: ['A', 'B', 'D', 'E', 'F', 'C']


### How to use:

1.  **Modify the `graph` dictionary** in the first code cell to define your own graph structure. The keys are nodes, and the values are lists of their adjacent nodes.
2.  **Run the cells** sequentially.
3.  When prompted, **enter a starting node** (e.g., 'A', 'B', 'C') for both BFS and DFS to see the traversal output.